In [1]:
import pandas as pd
import urllib
import requests
import numpy as np
import os
import datetime
from bs4 import BeautifulSoup

# URLを取得し、解析する #
def get_and_bs_html(dic):
    html = requests.get(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "zero-one":
        bs = BeautifulSoup(html.content,"html.parser")
    else:
        bs = BeautifulSoup(html.text,"html.parser")
    return bs

# 新着記事かどうかの判断基準を決める #
def select_criterion(dic,df):
    df_sorted_by_source = df[df["Source"]==dic["Name"]]
    if (dic["Name"] == "Infra") | (dic["Name"] == "Wantedly"):
        latest_date = list(df_sorted_by_source[-1:]["Date"].values)[0]
        criterion = df_sorted_by_source[df_sorted_by_source["Date"]==latest_date][:1]["Title"].values # latest title
    else:
        criterion = list(df_sorted_by_source["Title"]) # titles_stored
    return criterion, df_sorted_by_source

# サイトによって異なるスクレイピング関数 #
def search_new_article(dic,df):
    # 前準備 #
    columns = ["Title","URL","Summary","Source","Date"]
    today = datetime.date.today().strftime("%Y/%m/%d")
    bs = get_and_bs_html(dic)
    criterion,df_sorted_by_source = select_criterion(dic,df)
    # 本番 #
    if dic["Name"] == "キャリアバイト": # キャリアバイト用
        all_articles = bs.find_all("a",{"class":"tapArea informClick"})
        for article in all_articles:
            avalibility = article.find("span",{"class":"newLabel"})
            if avalibility == None: # Newのラベルがなければbreak
                break
            else:
                title = article.find("h3",{"class":"summaryList_ttl"})
                for e in title:
                    if (e.string != "NEW") & (len(e.string)>1):
                        title_ = e.string.strip("\n ") # タイトルを特定
                    else:
                        continue
                if title_ not in criterion: # 新着記事であれば
                    url = dic["URL"]+article.get("href") # URLを特定
                    # summaryを特定 #
                    summary = article.find("div",{"class":"summaryList_lead"}).string
                    print(title_)
                    print(url)
                    print(summary)
                    se = pd.Series([title_,url,summary,"キャリアバイト",today],index=columns)
                    df = df.append(se,ignore_index=True)
                else:
                    continue
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "Infra": # Infra用
        all_articles = bs.find_all("div",{"class":"col s12 m6"})
        for article in all_articles:
            title_and_url = article.find("h3",{"class":"intern-title"})
            title = title_and_url.find("a").string.strip("\n ") # タイトルを特定
            # 新着記事がなければ #
            if title == criterion:
                break
            # 新着記事であれば #
            else:
                # URLを取得 #
                url = dic["URL"]+title_and_url.find("a").get("href")
                # summaryを取得 #
                summary = article.find("td",{"class":"job-type"}).string
                print(title)
                print(url)
                print(summary)
                se = pd.Series([title,url,summary,"Infra",today],index=columns)
                df = df.append(se,ignore_index=True)
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "zero-one": # ゼロワン用
        all_articles = bs.find_all("section",{"class":"l-common-joblistBox"})
        for article in all_articles:
            avalibility = article.find("div",{"class":"l-common-joblist_fav_sp display-none_pc"}).find("span").string
            if avalibility != "お気に入り": # 募集中でなければbreak
                break
            else:
                title_and_url = article.find("div",{"class":"l-common-joblist_text"})
                # タイトルを取得 #
                title = title_and_url.find("h4").string.strip("\n ")
                # 募集中でありかつ新着であれば #
                if title not in criterion:
                    # URLを取得 #
                    url = dic["URL"] + title_and_url.find("a").get("href")
                    summary_and_loc = article.find_all("dl",{"class":"display-none_sp"})
                    # summaryを取得 #
                    summary = summary_and_loc[1].find("dd").string.strip("\n ")
                    print(title)
                    print(url)
                    print(summary)
                    se = pd.Series([title,url,summary,"zero-one",today],index=columns)
                    df = df.append(se,ignore_index=True)
                else:
                    continue
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "Wantedly": # Wantedly用
        all_articles = bs.find_all("article",{"class":"projects-index-single"})
        for article in all_articles:
            title = article.find("h1",{"class":"project-title"}).find("a").string # タイトルを特定
            # 新着記事がなければ #
            if title == criterion:
                break
            # 新着記事であれば #
            else:
                # URLを取得 #
                url = dic["URL"]+article.find("h1",{"class":"project-title"}).find("a").get("href")
                # summaryを取得 #
                summary = article.find("p",{"class":"project-excerpt"}).string.strip("\n ")
                print(title)
                print(url)
                print(summary)
                se = pd.Series([title,url,summary,"Wantedly",today],index=columns)
                df = df.append(se,ignore_index=True)
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    return df
# サイトおよび条件の辞書 #
dic_career = {
    "Name":"キャリアバイト",
    "URL":"https://careerbaito.com",
    "Conditions":"/search/38/305/316"
}
dic_infra = {
    "Name":"Infra",
    "URL":"https://www.in-fra.jp/",
    "Conditions":"long-internships?occupation%5B%5D=1&occupation%5B%5D=7&area%5B%5D=1&area%5B%5D=4&area%5B%5D=5&area%5B%5D=2&industry%5B%5D=1&industry%5B%5D=4&industry%5B%5D=11&order=recent"
}
dic_zeroone = {
    "Name":"zero-one",
    "URL":"https://01intern.com",
    "Conditions":"/job/list.html?areas=13&jobTypes=6&jobTypes=2&businessTypes=1&businessTypes=4&businessTypes=8"
}
dic_wantedly = {
    "Name":"Wantedly",
    "URL":"https://www.wantedly.com/",
    "Conditions":"/projects?type=recent&page=1&occupation_types%5B%5D=jp__engineering&hiring_types%5B%5D=internship&fields%5B%5D=jp__data_scientist&locations%5B%5D=tokyo"
}
dic_list = [dic_career,dic_infra,dic_zeroone,dic_wantedly]

In [2]:
csv_path = os.path.join("datasets","Internship.csv")
df = pd.read_csv(csv_path,index_col=0)
df

,Title,URL,Summary,Source,Date
0,【技術で事業を創る】事業と一緒に成長したいエンジニアインターン！,https://careerbaito.com/job/32790,#東証一部上場連結子会社＃エンジニア＃WEB開発＃アプリ開発＃インフラ構築＃自然言語処理＃機械学習,キャリアバイト,2020/05/04
1,【経営コンサルタント社長直下】新規事業立ち上げ・BtoBマーケティング・営業企画のインターン,https://careerbaito.com/job/32562,＼新規事業立ち上げ／経営コンサルティングの企画・営業の裁量をすべてお任せます・自律的にジャッ...,キャリアバイト,2020/05/04
2,「美少女図鑑」WEBサイト制作インターン,https://www.in-fra.jp//long-internships/9742,WEBサイト制作,Infra,2020/05/04
3,フロントエンド／バックエンドエンジニア募集,https://www.in-fra.jp//long-internships/9728,フロントエンド／バックエンドエンジニア,Infra,2020/05/04
4,【未経験OK】営業・マーケ・CSを経験できるインターン！営業フリーランスを世の中に浸透させ業...,https://www.in-fra.jp//long-internships/9735,アウトバウンド営業 / マーケティング/CS,Infra,2020/05/04
5,環境・社会問題の解決に貢献！リサイクル業界の長期インターンシップ,https://01intern.com/job/2643.html,【環境問題に取り組みたい大学生におすすめ！生涯ためになるスキルを身につけよう】株式会社リジェ...,zero-one,2020/05/05
6,AIベンチャーの最先端の実務経験を獲得！文理共歓迎。本郷,https://01intern.com/job/1728.html,メタデータは、機械学習・ディープラーニングの技術を活かしたWebアプリ、ソフトウェアを開発し...,zero-one,2020/05/05
7,時給1200円！公認会計士・税理士志望の学生おすすめインターン募集,https://01intern.com/job/1550.html,永井圭介公認会計士・税理士事務所では、公認会計士・税理士のアシスタントをしてくれる学生インタ...,zero-one,2020/05/05
8,勤務時間が選べる！事務スキルが見につく、IT企業の長期インターン,https://01intern.com/job/1944.html,クリスタルメソッド株式会社は、AIを活用したシステム開発を行っている会社です。不良品検知のシ...,zero-one,2020/05/05
9,IT系コンサル業界志望必見！ ITコンサル社長の実践力を身に付ける,https://01intern.com/job/1126.html,【コンサルティング企業の社長の下で学ぶコンサルインターン】ブレイブ・システム・コンサルティン...,zero-one,2020/05/05


In [3]:
for target in dic_list:
    df = search_new_article(target,df=df)

キャリアバイトに新着求人はない
https://careerbaito.com/search/38/305/316
Infraに新着求人はない
https://www.in-fra.jp/long-internships?occupation%5B%5D=1&occupation%5B%5D=7&area%5B%5D=1&area%5B%5D=4&area%5B%5D=5&area%5B%5D=2&industry%5B%5D=1&industry%5B%5D=4&industry%5B%5D=11&order=recent
zero-oneに新着求人はない
https://01intern.com/job/list.html?areas=13&jobTypes=6&jobTypes=2&businessTypes=1&businessTypes=4&businessTypes=8
Wantedlyに新着求人はない
https://www.wantedly.com//projects?type=recent&page=1&occupation_types%5B%5D=jp__engineering&hiring_types%5B%5D=internship&fields%5B%5D=jp__data_scientist&locations%5B%5D=tokyo


In [4]:
df.to_csv(csv_path,encoding='utf_8_sig')

In [5]:
df

,Title,URL,Summary,Source,Date
0,【技術で事業を創る】事業と一緒に成長したいエンジニアインターン！,https://careerbaito.com/job/32790,#東証一部上場連結子会社＃エンジニア＃WEB開発＃アプリ開発＃インフラ構築＃自然言語処理＃機械学習,キャリアバイト,2020/05/04
1,【経営コンサルタント社長直下】新規事業立ち上げ・BtoBマーケティング・営業企画のインターン,https://careerbaito.com/job/32562,＼新規事業立ち上げ／経営コンサルティングの企画・営業の裁量をすべてお任せます・自律的にジャッ...,キャリアバイト,2020/05/04
2,「美少女図鑑」WEBサイト制作インターン,https://www.in-fra.jp//long-internships/9742,WEBサイト制作,Infra,2020/05/04
3,フロントエンド／バックエンドエンジニア募集,https://www.in-fra.jp//long-internships/9728,フロントエンド／バックエンドエンジニア,Infra,2020/05/04
4,【未経験OK】営業・マーケ・CSを経験できるインターン！営業フリーランスを世の中に浸透させ業...,https://www.in-fra.jp//long-internships/9735,アウトバウンド営業 / マーケティング/CS,Infra,2020/05/04
5,環境・社会問題の解決に貢献！リサイクル業界の長期インターンシップ,https://01intern.com/job/2643.html,【環境問題に取り組みたい大学生におすすめ！生涯ためになるスキルを身につけよう】株式会社リジェ...,zero-one,2020/05/05
6,AIベンチャーの最先端の実務経験を獲得！文理共歓迎。本郷,https://01intern.com/job/1728.html,メタデータは、機械学習・ディープラーニングの技術を活かしたWebアプリ、ソフトウェアを開発し...,zero-one,2020/05/05
7,時給1200円！公認会計士・税理士志望の学生おすすめインターン募集,https://01intern.com/job/1550.html,永井圭介公認会計士・税理士事務所では、公認会計士・税理士のアシスタントをしてくれる学生インタ...,zero-one,2020/05/05
8,勤務時間が選べる！事務スキルが見につく、IT企業の長期インターン,https://01intern.com/job/1944.html,クリスタルメソッド株式会社は、AIを活用したシステム開発を行っている会社です。不良品検知のシ...,zero-one,2020/05/05
9,IT系コンサル業界志望必見！ ITコンサル社長の実践力を身に付ける,https://01intern.com/job/1126.html,【コンサルティング企業の社長の下で学ぶコンサルインターン】ブレイブ・システム・コンサルティン...,zero-one,2020/05/05
